In [ ]:
import os
import kagglehub
import cv2
import numpy as np





# Lütfen bu linkten çalıştırabilirseniz harika olur , projeyi colab üzerinden yaptım.

# Colab Linki
# https://colab.research.google.com/drive/11IT_cXQ67DwkCGv2wXqwescwJJ6RncOV?usp=sharing
# Çalışma zamanı > Çalışma zamanı türünü değiştir > v2-8 TPU seçeneğini seçin.
# Ben projeyi Google Colab üzerinden yaptım ve bu modeli kullandım.


# Kagglehub üzerinden veri setini indiriyoruz
veri_seti_yolu = kagglehub.dataset_download("rrebirrth/animals-with-attributes-2")


# İndirilen veri setinin dosya sıraları
ana_dizin = "/root/.cache/kagglehub/datasets/rrebirrth/animals-with-attributes-2/versions/1/Animals_with_Attributes2/JPEGImages/"

# Sınıflandırılıcak hayvanlar
istenen_hayvanlar = ["collie", "dolphin", "elephant", "fox", "moose",
                     "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"]

# Gereksiz sınıfları silelim
print("Gereksiz hayvanlar siliniyor ...")
for klasor_yolu, alt_klasorler, _ in os.walk(ana_dizin, topdown=False):
    for alt_klasor in alt_klasorler:
        klasor_tam_yolu = os.path.join(klasor_yolu, alt_klasor)
        if not any(hayvan in klasor_tam_yolu for hayvan in istenen_hayvanlar):
            print(f"{klasor_tam_yolu} klasörü siliniyor ...")
             # Alt klasördeki dosyaları silme işlemi
            for dosya in os.listdir(klasor_tam_yolu):
                dosya_tam_yolu = os.path.join(klasor_tam_yolu, dosya)
                if os.path.isfile(dosya_tam_yolu):
                    os.remove(dosya_tam_yolu)
                elif os.path.isdir(dosya_tam_yolu):
                    os.rmdir(dosya_tam_yolu)  # Alt klasörleri sil
            os.rmdir(klasor_tam_yolu)  # Son olarak dizini sil


# Fazlalık resimleri silelim , her sınıftan 650 tane olucak
print("Her sınıftan yalnızca 650 resim bırakılıyor...] \n")
for hayvan in istenen_hayvanlar:
    klasor_yolu = os.path.join(ana_dizin, hayvan)

    # Eğer klasör mevcutsa işlem yap
    if os.path.exists(klasor_yolu):
        dosyalar = os.listdir(klasor_yolu)
        tam_yollar = sorted([os.path.join(klasor_yolu, dosya) for dosya in dosyalar])
        print(f"{hayvan} sınıfı için toplam resim sayısı: {len(dosyalar)}")


        # 650'den fazla resmi sil 
        for fazlalik_yol in tam_yollar[650:]:
            print(f"{fazlalik_yol} siliniyor...")
            os.remove(fazlalik_yol)  # Fazlalık resmi sil

print("\nVeri kümesi temizleme işlemi başarıyla tamamlandı!")

# Resimleri boyutlandırma ve normalize etme fonksiyonu
def boyutayarlama_ve_normalizasyon(resim_yolu, size=(128, 128)):
  img = cv2.imread(resim_yolu)  # Resmi okuma
  img_boyut = cv2.resize(img, size)  # Resmi boyutlandırma
  img_normalizasyon = img_boyut / 255.0  # Resmi normalize etme
  return img_normalizasyon

X = []  # Resimler
y = []  # Etiketler
import numpy as np
for index,hayvan in enumerate(istenen_hayvanlar):
  klasor_yolu = os.path.join(ana_dizin,hayvan)
  for resim in os.listdir(klasor_yolu):
    resim_yolu = os.path.join(klasor_yolu,resim)
    resim_yolu = boyutayarlama_ve_normalizasyon(resim_yolu)# Resmi oku boyutlandır ve normalize et
    X.append(resim_yolu) # Resim ekle
    y.append(index)  # Etiket ekle

X = np.array(X)
y = np.array(y)

# Eğitim ve test verilerini ayrıyoruz
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,random_state=42,test_size=0.3)

# onehot encoding işlemi
from tensorflow.keras.utils import to_categorical

Y_train = to_categorical(Y_train, num_classes=len(istenen_hayvanlar))
Y_test = to_categorical(Y_test, num_classes=len(istenen_hayvanlar))


from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Veri artırma işlemleri
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Veri arttırma 
datagen.fit(X_train)

from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam

# VGG16 modelini yükleyelim (pre-trained ImageNet ağırlıkları ile)
vgg_base = VGG16(weights="imagenet", include_top=False, input_shape=(128, 128, 3))

# VGG katmanlarını donduruyoruz artık weightler eğitilmeyecek
for layer in vgg_base.layers:
    layer.trainable = False

# Son 4 katmanı eğitilebilir yapıyoruz
for layer in vgg_base.layers[-4:]:
    layer.trainable = True


# Kendi sınıflandırma katmanlarımızı ekleyelim
x = vgg_base.output
x = Flatten()(x)  # Düzleştirme işlemi
x = Dense(256, activation="relu")(x)  # Tam bağlı katman
x = Dropout(0.5)(x)  # Dropout yaparak overfitting engelliyoruz
output = Dense(len(istenen_hayvanlar), activation="softmax")(x)  # Çıktı katmanı




# Yeni modeli oluşturma
vgg_model = Model(inputs=vgg_base.input, outputs=output)

# Modeli derleme
vgg_model.compile(optimizer=Adam(learning_rate=0.0001),  # Metriklerimiz
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])



epochs = 20  # Eğitim boyutu
batch_size = 128 # Batch boyutu


# Modeli eğitme işlemleri
history = vgg_model.fit(datagen.flow(X_train, Y_train, batch_size=batch_size),# Test verileri
                        epochs=epochs,
                        validation_data=(X_test, Y_test), # Doğrulama verisi
                        steps_per_epoch=X_train.shape[0] // batch_size)


# Loss ve acc değerlerini ekrana yazdırmaya yarar
score = vgg_model.evaluate(X_test,Y_test,verbose = 0)
print("Test Loss : ",score[0])
print("Test Accuracy : ",score[1])

import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='Eğitim Doğruluk')
plt.plot(history.history['val_accuracy'], label='Test Doğruluk')
plt.title("VGG16 Modeli Sonuçları")
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')        # Görselleştirme işlemleri
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Eğitim Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title("VGG16 Modeli Sonuçları")
plt.xlabel('Epoch')
plt.ylabel('Loss')        # Görselleştirme işlemleri
plt.legend()
plt.show()


# Test resimlerini manipüle etme ve kaydetme
def manipulate_and_save_images(test_images):
    manipulated_images = get_manipulated_images(test_images)  # Manipüle edilmiş görüntüler oluştur
    manipulated_images = np.array(manipulated_images)  # NumPy dizisine dönüştür
    return manipulated_images

# Manipüle edilmiş resimleri oluşturma
def get_manipulated_images(test_images):
    manipulated_images = []
    for image in test_images:
        noise = np.random.normal(0, 0.1, image.shape)  # Gürültü ekleme
        noisy_image = np.clip(image + noise, 0, 1)  # Değerleri 0-1 arasında sınırlama
        manipulated_images.append(noisy_image)
    return manipulated_images

# Renk sabitliği uygulama
def apply_white_balance_on_manipulated_images(manipulated_images):
    wb_images = []
    for image in manipulated_images:
        wb_image = image  # Burada gerçek renk sabitliği fonksiyonu uygulanabilir
        wb_images.append(wb_image)
    return np.array(wb_images)

# Modelin test seti ile değerlendirilmesi
def evaluate_model(model, images, labels):
    return model.evaluate(images, labels, verbose=0)[1]

# Manipüle edilmiş ve renk sabitliği uygulanmış test setlerini değerlendirme
manipulated_images = manipulate_and_save_images(X_test)  # Manipüle edilmiş test seti
manipulated_scores = evaluate_model(vgg_model, manipulated_images, Y_test)  # Manipüle edilmiş test başarısı
wb_images = apply_white_balance_on_manipulated_images(manipulated_images)  # Renk sabitliği uygulanmış test seti
wb_scores = evaluate_model(vgg_model, wb_images, Y_test)  # Renk sabitliği uygulanmış test başarısı

# Sonuçları karşılaştırma ve raporlama
def compare_and_report_results(original_scores, manipulated_scores, wb_scores):
    print("Orijinal Test Seti Başarı Oranı:", original_scores)
    print("Manipüle Edilmiş Test Seti Başarı Oranı:", manipulated_scores)
    print("Renk Sabitliği Uygulanmış Test Seti Başarı Oranı:", wb_scores)

    if wb_scores > manipulated_scores:
        print("Renk sabitliği iyileştirme sağladı!")
    else:
        print("Başarıda önemli bir fark bulunamadı.")

# Orijinal test seti başarısı
original_scores = evaluate_model(vgg_model, X_test, Y_test)

# Sonuçları karşılaştır ve raporla
compare_and_report_results(original_scores, manipulated_scores, wb_scores)
